In [1]:
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm
import ipyplot
import dlib

from utils import *

In [2]:
# # DEBUG
# facedetect_data = random.sample(facedetect_data, N_DEBUG)
# face_count = 0
# for model in facedetect_data:
#     if model.faces[0].probability != 0:
#         for face in model.faces:
#             face_count += 1
# print("{:,} faces in query debug results.".format(face_count))

In [3]:
facedetect_data = Model.find((Model.filename % "wedding*moose")).all()

face_count = 0
for model in facedetect_data:
    if model.faces[0].probability != 0:
        for face in model.faces:
            face_count += 1
print("{:,} faces in query results.".format(face_count))

1,751 faces in query results.


In [4]:
triton_client = grpcclient.InferenceServerClient(url=TRITON_SERVER_URL, verbose=False)

model_metadata = triton_client.get_model_metadata(
    model_name=DLIB_FACE_DESCRIPTOR_MODEL_NAME, model_version=MODEL_VERSION
)

model_config = triton_client.get_model_config(
    model_name=DLIB_FACE_DESCRIPTOR_MODEL_NAME, model_version=MODEL_VERSION
).config

input_names = [i.name for i in model_config.input]
output_names = [i.name for i in model_config.output]

In [5]:
n = 0
max_images = 8
results = []
for model in facedetect_data:
    if model.faces[0].probability != 0:
        if n == max_images:
            break
        else:
            try:
                json_items = model.dict()
                result = submit_to_dlib(
                    json_items,
                    input_names[0],
                    output_names,
                    request_id=json_items["filename"],
                )
                results.append(result)
                n += 1
            except Exception as e:
                print("There was an exception: {}".format(e))

descriptors = []
for result in results:
    for descriptor in result:
        descriptors.append(descriptor.as_numpy("face_descriptor"))

print("Example 128D descriptor vector:\n{}".format(descriptors[0]))

Example 128D descriptor vector:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
pbar = tqdm(
    total=len(facedetect_data),
    desc="Submitting photos to {} at {}".format(
        DLIB_FACE_DESCRIPTOR_MODEL_NAME, TRITON_SERVER_URL
    ),
)

descriptors = []
with ThreadPoolExecutor() as executor:
    for chunk in chunked(facedetect_data, THREAD_CHUNKS):
        futures = []
        for model in chunk:
            pbar.update()
            if model.faces[0].probability != 0:
                json_items = model.dict()
                futures.append(
                    executor.submit(
                        submit_to_dlib,
                        json_items,
                        input_names[0],
                        output_names,
                        request_id=json_items["filename"],
                    )
                )
        for future in as_completed(futures):
            try:
                infer_results = future.result()
                for infer_result in infer_results:
                    descriptors.append(infer_result.as_numpy("face_descriptor"))

            except Exception as e:
                print("There was an exception: {}".format(e))

In [ ]:
idx = 0
for model in facedetect_data:
    if model.faces[0].probability != 0:
        for i, face in enumerate(model.faces):
            model.faces[i].descriptor = str(descriptors[idx].tobytes())
            model.save()
            idx += 1

In [ ]:
dlib_descriptors = [dlib.vector(descriptor) for descriptor in descriptors]

In [ ]:
dlib.chinese_whispers_clustering?

In [ ]:
print("calculating clusters for {:,} faces.".format(len(dlib_descriptors)))
labels = dlib.chinese_whispers_clustering(dlib_descriptors, 0.5)
print("done.")

In [ ]:
for model in facedetect_data:
    if len(model.faces) > 1:
        tmp = model
tmp.dict()

___